In [ ]:
import yaml
import numpy as np
import pandas as pd
import networkx as nx

# We want to import stuff from parent directory
import sys
import os
sys.path.append(os.path.join(sys.path[0], '..'))

from src import data_preperation
from src import db_requests

### VGAE
import os.path as osp
import argparse
import torch

import orca

In [ ]:
##### CONFIGURATIONS #####

with open("../db.yaml", 'r') as dbfi:
    db_credentials = yaml.safe_load(dbfi)

city = 'braunschweig'
save_path = '../data'

In [ ]:
streetgraph = db_requests.getStreetGraph_forCity(db_credentials, city)

In [ ]:
streetgraph

## POIs

In [ ]:
poi_df = db_requests.getPOIs(db_credentials)

In [ ]:
poi_df

-- restaurants and cafes <br>
-- bakery, restaurant, fast_food, cafe, pub<br>
<br>
-- daily POIs <br>
-- supermarket, pharmacy, bank, kindergarten, school, sports_centre, post_office<br>

In [ ]:
# Iterate through each street and count poi groups
for index, row in poi_df.iterrows():
    eating_count = 0
    daily_count = 0
    stop_count = 0
    
    pois = row['poi_id']
    # Iterate over all near POIs and count groups
    for poi in pois:
        if poi in ['bakery', 'restaurant', 'fast_food', 'cafe', 'pub']:
            eating_count = eating_count + 1
        elif poi in ['supermarket', 'pharmacy', 'bank', 'kindergarten', 'school', 'sports_centre', 'post_office', 'parking']:
            daily_count = daily_count + 1
        elif poi in ['stop_position']:
            stop_count = stop_count + 1
    
    # Add to df
    poi_df.at[index,'eating_count'] = eating_count
    poi_df.at[index,'daily_count'] = daily_count
    poi_df.at[index,'stop_count'] = stop_count
    

In [ ]:
# Delete columns
poi_df = poi_df.drop('array_agg', 1)
poi_df = poi_df.drop('poi_id', 1)

In [ ]:
streetgraph = streetgraph.merge(poi_df, on='id', how='left')
# Some streets do not have pois, fill them with 0
streetgraph = streetgraph.fillna(0)

## Graph

In [ ]:
# Create networkz graph
G = data_preperation.construct_graph(streetgraph)

### Get Edge Degrees

In [ ]:
# This lists save the amount of edges 
first_deg_list = []
second_deg_list = []
third_deg_list = []

# Iterate over each street and count edges
for index, row in streetgraph.iterrows():
    row_target = row['target']
    
    # Get next streets
    out_edges_1st = list(G.out_edges(row_target))
    amount_1st = len(out_edges_1st)
    first_deg_list.append(amount_1st)
    
    # Get second degree streets
    out_edges_2nd = []    
    for edge in out_edges_1st:
        out_edges_2nd = out_edges_2nd + list(G.out_edges(edge[1]))
    amount_2nd = len(out_edges_2nd)
    second_deg_list.append(amount_2nd)
    
    # Get third degree streets
    out_edges_3rd = []
    for edge in out_edges_2nd:
        out_edges_3rd = out_edges_3rd + list(G.out_edges(edge[1]))
    amount_3rd = len(out_edges_3rd)
    third_deg_list.append(amount_3rd)

In [ ]:
# Add to Table
#streetgraph['edges_first_deg'] = first_deg_list
#streetgraph['edges_second_deg'] = second_deg_list
#streetgraph['edges_third_deg'] = third_deg_list

In [ ]:
# Create a Crossings List
# If next degree > 2 then its a crossing
crossing_list = [1 if n>2 else 0  for n in  first_deg_list]

In [ ]:
# Add to table
streetgraph['crossing'] = crossing_list

## Graphembedding ##

In [ ]:
# Get Linegraph
L = nx.line_graph(G)
print(len(L.nodes))

In [ ]:

undirected_zero_start_L = nx.Graph(L)
undirected_zero_start_L = nx.convert_node_labels_to_integers(undirected_zero_start_L)

for node in undirected_zero_start_L:
    if undirected_zero_start_L.has_edge(node, node):
        undirected_zero_start_L.remove_edge(node, node)

counts = orca.orbit_counts("node", 5, undirected_zero_start_L)

In [ ]:
embeddings = np.array(counts)

In [ ]:
embed_size=embeddings.shape[1]

## Combine all feature extractions ##

Create a numpy array with shape (N, #road_feats + #graph_embedding).

In [ ]:
# Features : dist_citycentre, max_speed, type, length, graph_embedding
streetgraph_encoded = pd.get_dummies(streetgraph, columns=["type"])
streetgraph_encoded.head()

In [ ]:
# Iterate through dataframe and add the embeddings

# Create np array with columns: id, 
N = len(streetgraph_encoded)
feats_mx = np.empty([N, embed_size+len(streetgraph_encoded.columns)])

for index, row in streetgraph_encoded.iterrows():
    # The dict key is a combination of source and target id
    source = row['source']
    target = row['target']
    node_id = (source, target)
    #print(street_id)
    
    # Find the right index 
    idx = list(L.nodes).index(node_id)

    # Get the embedding
    emb = embeddings[idx]
    
    # Add embedding and streefeats to a numpy array
    newrow = np.concatenate((row.values, emb))
    feats_mx[index] = newrow

# Drop the target and source columns
feats_mx = np.delete(feats_mx, np.s_[3:5], axis=1)   

In [ ]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
feats_mx[0,:]

In [ ]:
feats_mx.shape

In [ ]:
# Save
np.save(save_path + f'/feats_mx_ablation1_{city}.npy', feats_mx)